In [2]:
# hydrological packages
import hydroeval as he
from hydrotools.nwm_client import utils 

# basic packages
import numpy as np
import pandas as pd
import os
import pyarrow as pa
import pyarrow.parquet as pq
import bz2file as bz2

# system packages
from progressbar import ProgressBar
from datetime import datetime, date
import datetime
import pickle as pkl
import warnings
warnings.filterwarnings("ignore")
import platform
import time

# data analysi packages
from scipy import optimize
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

# deep learning packages
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

#Shared/Utility scripts
import sys
import boto3
import s3fs
sys.path.insert(0, '..') #sys allows for the .ipynb file to connect to the shared folder files
from shared_scripts import lstm_dataprocessing

#load access key
HOME = os.path.expanduser('~')
KEYPATH = "NWM_ML/AWSaccessKeys.csv"
ACCESS = pd.read_csv(f"{HOME}/{KEYPATH}")

#start session
SESSION = boto3.Session(
    aws_access_key_id=ACCESS['Access key ID'][0],
    aws_secret_access_key=ACCESS['Secret access key'][0],
)
S3 = SESSION.resource('s3')
#AWS BUCKET information
BUCKET_NAME = 'streamflow-app-data'
BUCKET = S3.Bucket(BUCKET_NAME)

#s3fs
fs = s3fs.S3FileSystem(anon=False, key=ACCESS['Access key ID'][0], secret=ACCESS['Secret access key'][0])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


modelname = "LSTM"

print(f"{modelname} development script")

Device: cuda
LSTM development script


## Next Steps
* establish lookback
* figure out how to randomize training data
* add new dataset
* import Evaluation_funcs

In [3]:
#Get streamstats data 
datapath = f"{HOME}/NWM_ML/Data/input"
file = "Streamstats.csv"
filepath = f"{datapath}/{file}"
try:
    StreamStats = pd.read_csv(filepath)
except:
    print("Data not found, retreiving from AWS S3")
    if not os.path.exists(datapath):
        os.makedirs(datapath, exist_ok=True)
    key = 'Streamstats/Streamstats.csv'      
    S3.meta.client.download_file(BUCKET_NAME, key,filepath)
    StreamStats = pd.read_csv(filepath)

#Get processed training data 
datapath = f"{HOME}/NWM_ML/Data/Processed"
file = "raw_training_data.parquet"
filepath = f"{datapath}/{file}"
try:
    df = pd.read_parquet(filepath)
except:
    print("Data not found, retreiving from AWS S3")
    if not os.path.exists(datapath):
        os.makedirs(datapath, exist_ok=True)
    key = "NWM_ML"+datapath.split("NWM_ML",1)[1]+'/'+file       
    S3.meta.client.download_file(BUCKET_NAME, key,filepath)
    df = pd.read_parquet(filepath)

df.pop('Unnamed: 0')
df['station_id'] = df['station_id'].astype('str')
df.head()

,station_id,Lat,Long,Drainage_area_mi2,Mean_Basin_Elev_ft,Perc_Forest,Perc_Develop,Perc_Imperv,Perc_Herbace,Perc_Slop_30,Mean_Ann_Precip_in,datetime,flow_cfs,s1,s2,storage,swe,NWM_flow,DOY
0,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-10-28,78.55521,-0.891007,-0.453991,0.0,1.2,55.0,301
1,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-10-29,98.61146,-0.891007,-0.453991,0.0,1.2,55.0,302
2,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-10-30,97.60208,-0.891007,-0.453991,0.0,1.1,54.0,303
3,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-10-31,99.33125,-0.891007,-0.453991,0.0,1.2,54.0,304
4,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,34.8,2010-11-01,95.76354,-0.998630,0.052336,0.0,1.2,54.0,305


### Dataprocessing
* Editing the features based on the feature importance
* Remove headwater stations from dataset
* make sure dates are in datetime format

In [4]:
#remove headwater stations
headwater_stations = ['10011500', # Bear River headwaters before WY state line
                      '10109000', # Logan River above dams
                      '10113500', # HW Blacksmith fork
                      '10128500', # Upper Weber above Oakley
                      '10131000', #Chalk creek before Weber - lots of upstream irrigation, potentially include
                        '10146400', #Currant Creek above Mona Reservoir - lots of upstream irrigation, potentially include
                        '10150500', #Spanish fork after diamond fork - potentially include because of 6th water diversion CUP
                        '10154200', #Upper Provo river after confluence of N/S forks - potentially include because of duchense tunnel water diversion CUP
                        '10172700', #Vernon creek 2 ranges west of Utah Lake, shouldnt be included because not in GSL basin 
                        '10172800', #Willow creek west of Gransville,  shouldnt be included because does not make it to GSL
                          '10172952'] #Dunn creek in Raft River Range, shouldnt be included because drains to bonnevile salt flats 
df = df[~df['station_id'].isin(headwater_stations)]

#convert dates to datetime format
df.datetime = pd.to_datetime(df.datetime)

# #reset index to clean up df
df.reset_index( inplace =  True, drop = True)

df.head()

,station_id,Lat,Long,Drainage_area_mi2,Mean_Basin_Elev_ft,Perc_Forest,Perc_Develop,Perc_Imperv,Perc_Herbace,Perc_Slop_30,Mean_Ann_Precip_in,datetime,flow_cfs,s1,s2,storage,swe,NWM_flow,DOY
0,10105900,41.57549,-111.85522,180.0,6700.0,20.5,1.01,0.0653,18.9,44.2,33.2,1992-10-01,9.250000,-0.891007,-0.453991,0.0,0.0,37.0,275
1,10105900,41.57549,-111.85522,180.0,6700.0,20.5,1.01,0.0653,18.9,44.2,33.2,1992-10-02,8.654167,-0.891007,-0.453991,0.0,0.0,36.0,276
2,10105900,41.57549,-111.85522,180.0,6700.0,20.5,1.01,0.0653,18.9,44.2,33.2,1992-10-03,9.466667,-0.891007,-0.453991,0.0,0.0,36.0,277
3,10105900,41.57549,-111.85522,180.0,6700.0,20.5,1.01,0.0653,18.9,44.2,33.2,1992-10-04,11.833333,-0.891007,-0.453991,0.0,0.0,36.0,278
4,10105900,41.57549,-111.85522,180.0,6700.0,20.5,1.01,0.0653,18.9,44.2,33.2,1992-10-05,10.195833,-0.891007,-0.453991,0.0,0.0,36.0,279


## Data scaling

Scaling your data for ML is done for all types of applications and helps the model converge faster. 
If there is no scaling performed, the model will essentially be forced to think certain features are more important than others, rather than being able to learn those things.

There are different ways you can scale the data, such as min-max or standard scaling; both of which are applicable for your model. 
If you know you have a fixed min and max in your dataset (e.g. images), you can use min-max scaling to fix your input and/or output data to be between 0 and 1.

For other applications where you do not have fixed bounds, standard scaling is useful.
This gives all of your features zero-mean and unit variance. Therefore, the distributions of inputs and/or outputs are the same, and the model can treat them as such.

The scaling for your outputs is important in defining the activation function for the output layer.
If you have min-max scaled outputs, you can use sigmoid, because it bounds the outputs to between 0 and 1. 
If you are using standard scaling for the outputs, you would want to be sure you use a linear activation function, because technically standard-scaled outputs are not bounded. 
The choice of output activation is important, and knowledge of how your outputs are scaled is important in determining which activation to use.

In [7]:
#Process DF to input tensors into lstm
#adding one feature at a time to observe changes in model performance
input_columns =['Drainage_area_mi2', 
                'Mean_Basin_Elev_ft',
                'Perc_Forest', 
                'Perc_Develop', 
                'Perc_Imperv', 
                'Perc_Herbace',
                'Perc_Slop_30', 
                'Mean_Ann_Precip_in',
                's1',
                's2', 
                'storage', 
                'swe', 
                'NWM_flow', 
                'DOY']

target = ['flow_cfs']

scalername_x = "All_feat_scaler_x.save"
scalername_y = "scaler_y.save"

modelpath = f"{HOME}/NWM_ML/Model/{modelname}"
x_scaler_path = f"{modelpath}/{scalername_x}"
y_scaler_path = f"{modelpath}/{scalername_y}"

lookback = 5
test_years = [2019,2020]

#split data into train/test - note, for LSTM there is not need to randomize the split - hmmmmmmmmm
#train_type either ratio or hold out
x_train_scaled_t, X_test_dic, y_train_scaled_t, y_test_dic = lstm_dataprocessing.Multisite_DataProcessing(df, 
                                                                                   input_columns, 
                                                                                   target, 
                                                                                   lookback, 
                                                                                   test_years, 
                                                                                   x_scaler_path, 
                                                                                   y_scaler_path, 
                                                                                   random_shuffle = True)

print(x_train_scaled_t.shape, y_train_scaled_t.shape)


torch.Size([121479, 5, 14]) torch.Size([121479])


## Train the model

* add lookback
* make the model a .py file and class when finalized. PyTorch only saves the weights of the layer/node, not the overall structure.

In [8]:
#Train the model
model_path = f"{HOME}/NWM_ML/Model/{modelname}"
start_time = time.time()

# Assuming you have your data loaded into NumPy arrays as x_train_scaled, y_train_scaled, x_test_scaled, y_test_scaled, x_scaled, y_scaled
# Hyperparameters
epochs = 5
batch_size = 50
learning_rate = 0.0001
decay = 1e-2
validation_split = 0.2
neurons = 300

# Create PyTorch datasets and dataloaders
train_dataset = TensorDataset(x_train_scaled_t, y_train_scaled_t)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False ) # might shuffle this

# Build the model, this needs to be a class, see LSTM-Tutorials
model = nn.LSTM(input_size=x_train_scaled_t.shape[2], hidden_size=neurons, bidirectional=True, batch_first=True).to(device)
fc = nn.Linear(neurons * 2, 1).to(device)  # Multiply by 2 for bidirectional LSTM

# Define loss and optimizer
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=decay) #

# Training loop 
for epoch in range(epochs):
    model.train()
    fc.train()
    total_loss = 0.0
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        

        output, _ = model(batch_x)
        output = fc(output[:, -1, :])
        loss = criterion(output, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}")

print('finish')
print("Run Time:" + " %s minutes " % ((time.time() - start_time)/60))
#save model - https://pytorch.org/tutorials/beginner/saving_loading_models.html, for a more efficient way to save... need to make the model as a class and we save the class...
if os.path.exists(model_path) == False:
    os.mkdir(model_path)
torch.save(model.state_dict(), f"{model_path}/{modelname}_model.pkl")
torch.save(fc.state_dict(), f"{model_path}/{modelname}_model_fc.pkl")

Epoch 1/5, Loss: 0.3212985474723602
Epoch 2/5, Loss: 0.31936510849759414
Epoch 3/5, Loss: 0.3193691355210763
Epoch 4/5, Loss: 0.3193684895680029
Epoch 5/5, Loss: 0.3193681662925232
finish
Run Time: 0.4882497310638428 minutes 


## Load the model for evaluation

In [ ]:
# Build and load the model
model_path = f"{HOME}/NWM_ML/Model/{modelname}"

# Build the model
model_P = nn.LSTM(input_size=x_train_scaled_t.shape[2], hidden_size=neurons, bidirectional=True, batch_first=True).to(device)
fc_P = nn.Linear(neurons * 2, 1).to(device)  # Multiply by 2 for bidirectional LSTM

#this requires the model structure to be preloaded
model_P.load_state_dict(torch.load(f"{model_path}/{modelname}_model.pkl"))
fc_P.load_state_dict(torch.load(f"{model_path}/{modelname}_model_fc.pkl"))


# Make a prediction for each location, save as compressed pkl file, and send predictions to AWS for use in CSES

In [ ]:
#get annual supply diffs
cfsday_AFday = 1.983
year = 2020

model_P = model_P.to(device)
fc_P = fc_P.to(device)


Preds_Dict = {}
for station_number in station_index_list.drop_duplicates():
  index = station_index_list == station_number
  X_test = x_test_temp_1[index]
  X_test_scaled_t = torch.Tensor(x_test_1_scaled[index]).unsqueeze(1)
  X_test_scaled_t = X_test_scaled_t.to(device)
  l = len(y_test_temp_1.values)
  y_test = torch.Tensor(np.array(y_test_temp_1.values).reshape(l,1))
  y_test = y_test.to(device)

  # Evaluation
  model_P.eval()
  with torch.no_grad():
    predictions_scaled, _ = model_P(X_test_scaled_t)
    predictions_scaled = fc_P(predictions_scaled[:, -1, :])

  # Invert scaling for actual
  predictions = scaler_y.inverse_transform(predictions_scaled.to('cpu').numpy())
  predictions[predictions<0] = 0
  predictions = pd.DataFrame(predictions, columns=[f"{modelname}_flow"])

  #save predictions, need to convert to NHDPlus reach - Need to add Datetime column and flow predictions
  #make daterange
  dates = pd.date_range(pd.to_datetime("2020-01-01"), periods=len(predictions)).strftime("%Y-%m-%d").tolist()
  predictions['Datetime'] = dates
    
  #get reach id for model eval
  nhdreach = utils.crosswalk(usgs_site_codes=station_number)
  nhdreach = nhdreach['nwm_feature_id'].iloc[0]

  #put columns in correct order
  cols = ['Datetime', f"{modelname}_flow"]
  predictions = predictions[cols]

  #save predictions to AWS so we can use CSES
  state = StreamStats['state_id'][StreamStats['NWIS_site_id'].astype(str)== station_number].values[0].lower()
  csv_key = f"{modelname}/NHD_segments_{state}.h5/{modelname[:3]}_{nhdreach}.csv"
  predictions.to_csv(f"s3://{BUCKET_NAME}/{csv_key}", index = False,  storage_options={'key': ACCESS['Access key ID'][0],
                           'secret': ACCESS['Secret access key'][0]})

  #Concat DFS and put into dictionary
  x_test_temp['nwm_feature_id'] = nhdreach
  Dfs = [predictions.reset_index(drop=True),x_test_temp[x_test_temp['station_id']==station_number].reset_index(drop=True)]
  Preds_Dict[station_number] = pd.concat(Dfs, axis=1)

  #reorganize columns
  Preds_Dict[station_number].pop('datetime')
  Preds_Dict[station_number].insert(1, f"{modelname}_flow", Preds_Dict[station_number].pop(f"{modelname}_flow"))
  Preds_Dict[station_number].insert(1, "NWM_flow", Preds_Dict[station_number].pop("NWM_flow"))
  Preds_Dict[station_number].insert(1, "flow_cfs", Preds_Dict[station_number].pop("flow_cfs"))
  Preds_Dict[station_number].insert(1, "nwm_feature_id", Preds_Dict[station_number].pop("nwm_feature_id"))
  Preds_Dict[station_number].insert(1, "station_id", Preds_Dict[station_number].pop("station_id"))

  #push data to AWS so we can use CSES
  
  
#save predictions as compressed pkl file
pred_path = f"{HOME}/NWM_ML/Predictions/Hindcast/{modelname}/{year}"
file_path = f"{pred_path}/{modelname}_predictions.pkl"
if os.path.exists(pred_path) == False:
  os.makedirs(pred_path)

with open(file_path, 'wb') as handle:
  pkl.dump(Preds_Dict, handle, protocol=pkl.HIGHEST_PROTOCOL)

In [ ]:
Preds_Dict['10105900']